In [3]:
from numpy.random import seed
seed(123)
import tensorflow as tf
#from tensorflow import set_random_seed
tf.random.set_seed(234)

import sklearn
from sklearn import datasets
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import decomposition
import scipy


from keras.models import Model, load_model
from keras.layers import Input, Dense, Layer, InputSpec
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers, activations, initializers, constraints, Sequential
from keras import backend as K
from keras.constraints import UnitNorm, Constraint

In [4]:
tf.__version__

'2.6.2'

# Generate random multi-dimensional correlated data

**Step 1**. Set the dimension of the data.

We set the dim small to clear understanding.

In [5]:
n_dim = 5

**Step 2.1.** Generate a positive definite symmetric matrix to be used as covariance to generate a random data.

This is a matrix of size n_dim x n_dim.

In [6]:
cov = sklearn.datasets.make_spd_matrix(n_dim, random_state=None)

**Step 2.2.** Generate a vector of mean for generating the random data.

This is an np array of size n_dim.

In [7]:
mu = np.random.normal(0, 0.1, n_dim)

**Step 3**. Generate the random data, `X`.

The number of samples for `X` is set as `n`.

In [8]:
n = 1000

X = np.random.multivariate_normal(mu, cov, n)
print("Type: ",type(X),"// and Shape: ", X.shape)

Type:  <class 'numpy.ndarray'> // and Shape:  (1000, 5)


**Step 4.** Split the data into train and test.

We split the data into train and test. The test will be used to measure the improvement in Autoencoder after tuning.

In [9]:
X_train, X_test = train_test_split(X, test_size=0.5, random_state=123)

# Data preprocessing

In [10]:
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)

X_test_scaled = scaler.transform(X_test)

In [11]:
X_train_scaled

array([[0.52681732, 0.62810095, 0.59284649, 0.52236607, 0.40995414],
       [0.41440755, 0.55001423, 0.63635182, 0.57413981, 0.30587165],
       [0.38378646, 0.14857921, 0.39851437, 0.34588993, 0.63374338],
       ...,
       [0.28371994, 0.4794476 , 0.48410551, 0.54245635, 0.59818998],
       [0.50510169, 0.76450508, 0.73271896, 0.69792451, 0.34080318],
       [0.3978237 , 0.50478323, 0.40141433, 0.67971132, 0.60012005]])

-----------

# PCA vs Single Layer Linear Autoencoder

### Fit Principal Component Analysis (PCA)

In [12]:
pca = decomposition.PCA(n_components=2)

pca.fit(X_train_scaled)

PCA(n_components=2)

### Fit Single Layer Linear Autoencoder

In [13]:
nb_epoch = 100
batch_size = 16
input_dim = X_train_scaled.shape[1] #num of predictor variables, 
encoding_dim = 2
learning_rate = 1e-3

encoder = Dense(encoding_dim, activation="linear", input_shape=(input_dim,), use_bias = True) 
decoder = Dense(input_dim, activation="linear", use_bias = True)

autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(decoder)

autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='sgd')
autoencoder.summary()

autoencoder.fit(X_train_scaled, X_train_scaled,
                epochs=nb_epoch,
                batch_size=batch_size,
                shuffle=True,
                verbose=0)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 15        
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


Compare and contrast the outputs.

### 1. Tied Weights

The weights on Encoder and Decoder are not the same.

In [14]:
w_encoder = np.round(autoencoder.layers[0].get_weights()[0], 2).T  # W in Figure 2.
w_decoder = np.round(autoencoder.layers[1].get_weights()[0], 2)  # W' in Figure 2.
print('Encoder weights \n', w_encoder)
print('Decoder weights \n', w_decoder)

Encoder weights 
 [[-0.48 -0.71  0.16 -0.06  0.46]
 [ 0.06 -0.   -0.31 -0.1  -0.18]]
Decoder weights 
 [[-0.56 -0.1  -0.44 -0.11  0.6 ]
 [ 0.16 -0.59 -0.87  0.36 -0.72]]


### 2. Weight Orthogonality
Unlike PCA weights, the weights on Encoder and Decoder are not orthogonal.

In [15]:
w_pca = pca.components_
np.round(np.dot(w_pca, w_pca.T), 3)

array([[1., 0.],
       [0., 1.]])

In [16]:
np.round(np.dot(w_encoder, w_encoder.T), 3)

array([[ 0.975, -0.155],
       [-0.155,  0.142]], dtype=float32)

In [17]:
np.round(np.dot(w_decoder, w_decoder.T), 3)

array([[ 0.889, -0.119],
       [-0.119,  1.779]], dtype=float32)

### 3. Uncorrelated Features
Unlike PCA features, i.e. Principal Scores, the Encoded features are correlated.

In [18]:
pca_features = pca.fit_transform(X_train_scaled)
np.round(np.cov(pca_features.T), 5)

array([[ 0.09899, -0.     ],
       [-0.     ,  0.01549]])

In [19]:
encoder_layer = Model(inputs=autoencoder.inputs, outputs=autoencoder.layers[0].output)
encoded_features = np.array(encoder_layer.predict(X_train_scaled))
print('Encoded feature covariance\n', np.cov(encoded_features.T))

Encoded feature covariance
 [[0.04867485 0.00448513]
 [0.00448513 0.00120718]]


### 4. Unit Norm

In [20]:
print('PCA weights norm, \n', np.sum(w_pca ** 2, axis = 1))
print('Encoder weights norm, \n', np.sum(w_encoder ** 2, axis = 1))
print('Decoder weights norm, \n', np.sum(w_decoder ** 2, axis = 1))

PCA weights norm, 
 [1. 1.]
Encoder weights norm, 
 [0.97529995 0.1421    ]
Decoder weights norm, 
 [0.8893 1.7786]


### Train Test Reconstruction Accuracy

In [21]:
train_predictions = autoencoder.predict(X_train_scaled)
print('Train reconstrunction error\n', sklearn.metrics.mean_squared_error(X_train_scaled, train_predictions))
test_predictions = autoencoder.predict(X_test_scaled)
print('Test reconstrunction error\n', sklearn.metrics.mean_squared_error(X_test_scaled, test_predictions))

Train reconstrunction error
 0.012426635119502481
Test reconstrunction error
 0.0123024308952595


--------

# Well-posed Autoencoder 
### Constraints for Autoencoder 
Optimizing Autoencoder using PCA principles

In [22]:
nb_epoch = 100
batch_size = 16
input_dim = X_train_scaled.shape[1] #num of predictor variables, 
encoding_dim = 2
learning_rate = 1e-3

### 1. Constraint: Tied weights

Make decoder weights equal to encoder.

In [23]:
class DenseTied(Layer):
    def __init__(self, units,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 tied_to=None,
                 **kwargs):
        self.tied_to = tied_to
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super().__init__(**kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.input_spec = InputSpec(min_ndim=2)
        self.supports_masking = True
                
    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]

        if self.tied_to is not None:
            self.kernel = K.transpose(self.tied_to.kernel)
            self._non_trainable_weights.append(self.kernel)
        else:
            self.kernel = self.add_weight(shape=(input_dim, self.units),
                                          initializer=self.kernel_initializer,
                                          name='kernel',
                                          regularizer=self.kernel_regularizer,
                                          constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)

    def call(self, inputs):
        output = K.dot(inputs, self.kernel)
        if self.use_bias:
            output = K.bias_add(output, self.bias, data_format='channels_last')
        if self.activation is not None:
            output = self.activation(output)
        return output

#### 1.1 Bias=False for Decoder

In [24]:
encoder = Dense(encoding_dim, activation="linear", input_shape=(input_dim,), use_bias = True) 
decoder = DenseTied(input_dim, activation="linear", tied_to=encoder, use_bias = False)

autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(decoder)

autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='sgd')
autoencoder.summary()

autoencoder.fit(X_train_scaled, X_train_scaled,
                epochs=3,
                batch_size=batch_size,
                shuffle=True,
                verbose=0)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 12        
_________________________________________________________________
dense_tied (DenseTied)       (None, 5)                 22        
Total params: 22
Trainable params: 12
Non-trainable params: 10
_________________________________________________________________


In [25]:
w_encoder = np.round(np.transpose(autoencoder.layers[0].get_weights()[0]), 3)
w_decoder = np.round(autoencoder.layers[1].get_weights()[0], 3)
print('Encoder weights\n', w_encoder)
print('Decoder weights\n', w_decoder)

Encoder weights
 [[-0.56   0.19   0.067  0.339 -0.074]
 [ 0.502  0.273  0.211 -0.181  0.05 ]]
Decoder weights
 [[-0.56   0.502]
 [ 0.19   0.273]
 [ 0.067  0.211]
 [ 0.339 -0.181]
 [-0.074  0.05 ]]


In [26]:
train_predictions = autoencoder.predict(X_train_scaled)
print('Train reconstrunction error\n', sklearn.metrics.mean_squared_error(X_train_scaled, train_predictions))
test_predictions = autoencoder.predict(X_test_scaled)
print('Test reconstrunction error\n', sklearn.metrics.mean_squared_error(X_test_scaled, test_predictions))

Train reconstrunction error
 0.2060967386664398
Test reconstrunction error
 0.2023363335559977


#### 1.2 Bias=True for Decoder

In [27]:
encoder = Dense(encoding_dim, activation="linear", input_shape=(input_dim,), use_bias = True) 
decoder = DenseTied(input_dim, activation="linear", tied_to=encoder, use_bias = True)

autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(decoder)

autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='sgd')
autoencoder.summary()

autoencoder.fit(X_train_scaled, X_train_scaled,
                epochs=nb_epoch,
                batch_size=batch_size,
                shuffle=True,
                verbose=0)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 2)                 12        
_________________________________________________________________
dense_tied_1 (DenseTied)     (None, 5)                 27        
Total params: 27
Trainable params: 17
Non-trainable params: 10
_________________________________________________________________


In [28]:
w_encoder = np.round(np.transpose(autoencoder.layers[0].get_weights()[0]), 3)
w_decoder = np.round(autoencoder.layers[1].get_weights()[1], 3)
print('Encoder weights\n', w_encoder)
print('Decoder weights\n', w_decoder)

Encoder weights
 [[ 0.145  0.458  0.353  0.311 -0.646]
 [ 0.381  0.637 -0.253 -0.88  -0.134]]
Decoder weights
 [[ 0.145  0.381]
 [ 0.458  0.637]
 [ 0.353 -0.253]
 [ 0.311 -0.88 ]
 [-0.646 -0.134]]


In [29]:
b_encoder = np.round(np.transpose(autoencoder.layers[0].get_weights()[1]), 3)
b_decoder = np.round(np.transpose(autoencoder.layers[1].get_weights()[0]), 3)
print('Encoder bias\n', b_encoder)
print('Decoder bias\n', b_decoder)

Encoder bias
 [0.027 0.067]
Decoder bias
 [0.458 0.425 0.391 0.32  0.653]


In [30]:
train_predictions = autoencoder.predict(X_train_scaled)
print('Train reconstrunction error\n', sklearn.metrics.mean_squared_error(X_train_scaled, train_predictions))
test_predictions = autoencoder.predict(X_test_scaled)
print('Test reconstrunction error\n', sklearn.metrics.mean_squared_error(X_test_scaled, test_predictions))

Train reconstrunction error
 0.00977973329933476
Test reconstrunction error
 0.009631461436930324


### 2. Constraint: Weights orthogonality.

In [31]:
class WeightsOrthogonalityConstraint (Constraint):
    def __init__(self, encoding_dim, weightage = 1.0, axis = 0):
        self.encoding_dim = encoding_dim
        self.weightage = weightage
        self.axis = axis
        
    def weights_orthogonality(self, w):
        if(self.axis==1):
            w = K.transpose(w)
        if(self.encoding_dim > 1):
            m = K.dot(K.transpose(w), w) - K.eye(self.encoding_dim)
            return self.weightage * K.sqrt(K.sum(K.square(m)))
        else:
            m = K.sum(w ** 2) - 1.
            return m

    def __call__(self, w):
        return self.weights_orthogonality(w)

#### 2.1 Encoder weight orthogonality

In [32]:
encoder = Dense(encoding_dim, activation="linear", input_shape=(input_dim,), use_bias=True, kernel_regularizer=WeightsOrthogonalityConstraint(encoding_dim, weightage=1., axis=0)) 
decoder = Dense(input_dim, activation="linear", use_bias = True)

autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(decoder)

autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='sgd')
autoencoder.summary()

autoencoder.fit(X_train_scaled, X_train_scaled,
                epochs=nb_epoch,
                batch_size=batch_size,
                shuffle=True,
                verbose=0)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 2)                 12        
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 15        
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


ValueError: in user code:

    /Users/nazmul/opt/anaconda3/envs/aeed-master/lib/python3.6/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /Users/nazmul/opt/anaconda3/envs/aeed-master/lib/python3.6/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/nazmul/opt/anaconda3/envs/aeed-master/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/nazmul/opt/anaconda3/envs/aeed-master/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/nazmul/opt/anaconda3/envs/aeed-master/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kw

In [33]:
w_encoder = autoencoder.layers[0].get_weights()[0]
print('Encoder weights dot product\n', np.round(np.dot(w_encoder.T, w_encoder), 2))

Encoder weights dot product
 [[1.94 0.62]
 [0.62 1.46]]


In [34]:
train_predictions = autoencoder.predict(X_train_scaled)
print('Train reconstrunction error\n', sklearn.metrics.mean_squared_error(X_train_scaled, train_predictions))
test_predictions = autoencoder.predict(X_test_scaled)
print('Test reconstrunction error\n', sklearn.metrics.mean_squared_error(X_test_scaled, test_predictions))

Train reconstrunction error
 0.5207403689841729
Test reconstrunction error
 0.5195074143423446


#### 2.2 Encoder and Decoder Weight orthogonality

In [35]:
encoder = Dense(encoding_dim, activation="linear", input_shape=(input_dim,), use_bias=True, kernel_regularizer=WeightsOrthogonalityConstraint(encoding_dim, weightage=1., axis=0)) 
decoder = Dense(input_dim, activation="linear", use_bias = True, kernel_regularizer=WeightsOrthogonalityConstraint(encoding_dim, weightage=1., axis=1))

autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(decoder)

autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='sgd')
autoencoder.summary()

autoencoder.fit(X_train_scaled, X_train_scaled,
                epochs=nb_epoch,
                batch_size=batch_size,
                shuffle=True,
                verbose=0)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 2)                 12        
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 15        
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


ValueError: in user code:

    /Users/nazmul/opt/anaconda3/envs/aeed-master/lib/python3.6/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /Users/nazmul/opt/anaconda3/envs/aeed-master/lib/python3.6/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/nazmul/opt/anaconda3/envs/aeed-master/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/nazmul/opt/anaconda3/envs/aeed-master/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/nazmul/opt/anaconda3/envs/aeed-master/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kw

In [36]:
w_encoder = autoencoder.layers[0].get_weights()[0]
print('Encoder weights dot product\n', np.round(np.dot(w_encoder.T, w_encoder), 2))

w_decoder = autoencoder.layers[1].get_weights()[0]
print('Decoder weights dot product\n', np.round(np.dot(w_decoder, w_decoder.T), 2))

Encoder weights dot product
 [[1.89 0.16]
 [0.16 0.29]]
Decoder weights dot product
 [[2.02 0.63]
 [0.63 1.33]]


### 3. : Uncorrelated Encoded features

In [37]:
class UncorrelatedFeaturesConstraint (Constraint):

    def __init__(self, encoding_dim, weightage=1.0):
        self.encoding_dim = encoding_dim
        self.weightage = weightage

    def get_covariance(self, x):
        x_centered_list = []

        for i in range(self.encoding_dim):
            x_centered_list.append(x[:, i] - K.mean(x[:, i]))

        x_centered = tf.stack(x_centered_list)
        covariance = K.dot(x_centered, K.transpose(x_centered)) / \
            tf.cast(x_centered.get_shape()[0], tf.float32)

        return covariance

    # Constraint penalty
    def uncorrelated_feature(self, x):
        if(self.encoding_dim <= 1):
            return 0.0
        else:
            output = K.sum(K.square(
                self.covariance - tf.math.multiply(self.covariance, K.eye(self.encoding_dim))))
            return output

    def __call__(self, x):
        self.covariance = self.get_covariance(x)
        return self.weightage * self.uncorrelated_feature(x)

In [38]:
encoder = Dense(encoding_dim, activation="linear", input_shape=(input_dim,), use_bias=True,
                activity_regularizer=UncorrelatedFeaturesConstraint(encoding_dim, weightage=1.))
decoder = Dense(input_dim, activation="linear", use_bias=True)

autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(decoder)

autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='sgd')
autoencoder.summary()

autoencoder.fit(X_train_scaled, X_train_scaled,
                epochs=nb_epoch,
                batch_size=batch_size,
                shuffle=True,
                verbose=0)

ValueError: Tensor-typed variable initializers must either be wrapped in an init_scope or callable (e.g., `tf.Variable(lambda : tf.truncated_normal([10, 40]))`) when building functions. Please file a feature request if this restriction inconveniences you.

In [39]:
encoder_layer = Model(inputs=autoencoder.inputs, outputs=autoencoder.layers[0].output)
encoded_features = np.array(encoder_layer.predict(X_train_scaled))
print('Encoded feature covariance\n', np.round(np.cov(encoded_features.T), 3))

IndexError: list index out of range

In [40]:
train_predictions = autoencoder.predict(X_train_scaled)
print('Train reconstrunction error\n', sklearn.metrics.mean_squared_error(X_train_scaled, train_predictions))
test_predictions = autoencoder.predict(X_test_scaled)
print('Test reconstrunction error\n', sklearn.metrics.mean_squared_error(X_test_scaled, test_predictions))

Train reconstrunction error
 1.585365375821467e-16
Test reconstrunction error
 1.632923197702365e-16


### 4. Constraint: Unit Norm

#### 4.1 Unit Norm constraint on Encoding Layer

In [41]:
encoder = Dense(encoding_dim, activation="linear", input_shape=(input_dim,), use_bias = True, kernel_constraint=UnitNorm(axis=0)) 
decoder = Dense(input_dim, activation="linear", use_bias = True)

autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(decoder)

autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='sgd')
autoencoder.summary()

autoencoder.fit(X_train_scaled, X_train_scaled,
                epochs=nb_epoch,
                batch_size=batch_size,
                shuffle=True,
                verbose=0)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 2)                 12        
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 15        
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


In [42]:
w_encoder = np.round(autoencoder.layers[0].get_weights()[0], 2).T  # W in Figure 2.
print('Encoder weights norm, \n', np.round(np.sum(w_encoder ** 2, axis = 1),3))

Encoder weights norm, 
 [1.001 0.998]


In [43]:
train_predictions = autoencoder.predict(X_train_scaled)
print('Train reconstrunction error\n', sklearn.metrics.mean_squared_error(X_train_scaled, train_predictions))
test_predictions = autoencoder.predict(X_test_scaled)
print('Test reconstrunction error\n', sklearn.metrics.mean_squared_error(X_test_scaled, test_predictions))

Train reconstrunction error
 0.013148496871253492
Test reconstrunction error
 0.01350254830832146


#### 4.2 Unit Norm constraint on both Encoding and Decoding Layer

In [44]:
encoder = Dense(encoding_dim, activation="linear", input_shape=(input_dim,), use_bias = True, kernel_constraint=UnitNorm(axis=0)) 
decoder = Dense(input_dim, activation="linear", use_bias = True, kernel_constraint=UnitNorm(axis=1))

autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(decoder)

autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='sgd')
autoencoder.summary()

autoencoder.fit(X_train_scaled, X_train_scaled,
                epochs=nb_epoch,
                batch_size=batch_size,
                shuffle=True,
                verbose=0)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 2)                 12        
_________________________________________________________________
dense_13 (Dense)             (None, 5)                 15        
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


In [45]:
w_encoder = np.round(autoencoder.layers[0].get_weights()[0], 2).T  # W in Figure 2.
w_decoder = np.round(autoencoder.layers[1].get_weights()[0], 2)  # W' in Figure 2.

print('Encoder weights norm, \n', np.round(np.sum(w_encoder ** 2, axis = 1),3))
print('Decoder weights norm, \n', np.round(np.sum(w_decoder ** 2, axis = 1),3))

Encoder weights norm, 
 [0.992 0.996]
Decoder weights norm, 
 [1.001 0.988]


In [46]:
train_predictions = autoencoder.predict(X_train_scaled)
print('Train reconstrunction error\n', sklearn.metrics.mean_squared_error(X_train_scaled, train_predictions))
test_predictions = autoencoder.predict(X_test_scaled)
print('Test reconstrunction error\n', sklearn.metrics.mean_squared_error(X_test_scaled, test_predictions))

Train reconstrunction error
 0.015571492515118673
Test reconstrunction error
 0.014894585747660598


----------

## Constraints put together

In [47]:
encoder = Dense(encoding_dim, activation="linear", input_shape=(input_dim,), use_bias = True, kernel_regularizer=WeightsOrthogonalityConstraint(encoding_dim, weightage=1., axis=0), kernel_constraint=UnitNorm(axis=0)) 
decoder = DenseTied(input_dim, activation="linear", tied_to=encoder, use_bias = False)

autoencoder = Sequential()
autoencoder.add(encoder)
autoencoder.add(decoder)

autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='sgd')
autoencoder.summary()

autoencoder.fit(X_train_scaled, X_train_scaled,
                epochs=nb_epoch,
                batch_size=batch_size,
                shuffle=True,
                verbose=0)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 2)                 12        
_________________________________________________________________
dense_tied_2 (DenseTied)     (None, 5)                 22        
Total params: 22
Trainable params: 12
Non-trainable params: 10
_________________________________________________________________


ValueError: in user code:

    /Users/nazmul/opt/anaconda3/envs/aeed-master/lib/python3.6/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /Users/nazmul/opt/anaconda3/envs/aeed-master/lib/python3.6/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/nazmul/opt/anaconda3/envs/aeed-master/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/nazmul/opt/anaconda3/envs/aeed-master/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/nazmul/opt/anaconda3/envs/aeed-master/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kw

In [48]:
train_predictions = autoencoder.predict(X_train_scaled)
print('Train reconstrunction error\n', sklearn.metrics.mean_squared_error(X_train_scaled, train_predictions))
test_predictions = autoencoder.predict(X_test_scaled)
print('Test reconstrunction error\n', sklearn.metrics.mean_squared_error(X_test_scaled, test_predictions))

Train reconstrunction error
 0.12785253364481158
Test reconstrunction error
 0.1251493887964157


-------------